In [1]:
# Install drake (if necessary) and set up the path.  
try:
  import pydrake
except ImportError:
  !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
  from jupyter_setup import setup_drake
  setup_drake()
# Note: On Google's Colaboratory, this will take a minute, but should only need to reinstall once every 12 hours.
# Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.

## A brief introduction to optimization

Optimization is finding the best solution to a problem subject to particular constraints. For example, driving the most efficient route from point A to point B given the amount of traffic on the roads. Mathematical optimization is written in a particular structure:

\begin{align*}
\min_{x} \ \ & f(x) & \\
\text{s.t.} \ \ & g_i(x) \leq 0, & i = 1, ..., m \\
         & h_j(x) = 0, & j = 1, ..., p \\
\end{align*}

Here, $m \geq 0$ and $p \geq 0$.

$f(x)$ is called the **objective function** or **cost**. <br>
$g_i(x)$ are the **inequality constraints**. <br>
$h_j(x)$ are the **equality constraints**.

There are a couple of important classes of optimization problems:

- Least Squares:
    Solving a problem of the form $$\min \|{Ax - b}\|^2_2.$$
    There is an analytic solution $$x^* = (A^TA)^{-1}A^Tb.$$
    
    
- Linear Programs (LPs):
    Solving a problem of the form \begin{align*}
                                        \min_{x} \ & c^Tx, \\
                                        \text{s.t.} \ \ & Ax \leq b.\\
                                    \end{align*}
    LPs have linear object functions and linear constraints.
    
    
- Quadratic Programs (QPs):
    Solving a problem of the form \begin{align*}
                                        \min_{x} \ & \frac{1}{2}x^TQx + c^Tx, \\
                                        \text{s.t.} \ \ &  Ax \leq b. \\
                                    \end{align*}
    QPs have quadratic object functions and linear constraints.

There are some other important classes, including Semidefinite Programs (SDPs), and (Mixed-)Integer Programs (MIP/IP).  We'll add more about those to these notes soon.  For now, you can check out the [course notes](https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-079-introduction-to-convex-optimization-fall-2009/lecture-notes/MIT6_079F09_lec01.pdf) from a version of MIT's 6.079 Intro to Convex Optimization for more information.

There are a variety of open-source and commercial optimization solvers that are really good at solving many types of optimization problems. Some examples include [SNOPT](https://web.stanford.edu/group/SOL/guides/sndoc7.pdf), [IPOPT](https://projects.coin-or.org/Ipopt), and [Gurobi](http://www.gurobi.com/index).

## How to use Mathematical Program

One area of optimization tools offered in pydrake is the MathematicalProgram interface.  MathematicalProgram is a class that abstracts many different useful types of optimization solvers.  This makes it so from one interface, you can access many different solvers.  The MathematicalProgram component of Drake is comparable to CVX or YALMIP in the MATLAB ecosystem, or JuMP, in the Julia ecosystem. To get a concise overview of which solvers are supported for which different types of optimization problems, check out [this chart](http://drake.mit.edu/doxygen_cxx/group__solvers.html).  

Here, we provide enough sample code to familiarize you with the tool and give you a sense of how to solve simple problems. If you're curious to dig deeper, take a look at the [Technical C++ Doxygen Documention](http://drake.mit.edu/doxygen_cxx/classdrake_1_1solvers_1_1_mathematical_program.html), and the [C++ source code](https://github.com/RobotLocomotion/drake/blob/master/solvers/mathematical_program.h).

In addition to the code snippets below, these two tips are also very useful:

- Once you construct a MathematicalProgram object, i.e. `mp = MathematicalProgram()`, the tab completion in your jupyter notebook can be very helpful.  

 --> For example, let's say you want to know if a MathematicalProgram can print how many decision variables currently exist.  Tab completing on `mp.` and scrolling through, you'll find `num_vars`.  Indeed `mp.num_vars()` will do the trick.
 
 --> Want to know which solver MP is currently using under the hood for a particular problem instance?
 
```python
result = Solve(mp) # must first solve the program (which forces a solver to be chosen)
solver = result.get_solver_id()
solver.name() # name will tab complete after creating a solver object
```
- An additional resource for how to use MathematicalProgram is the tests written for it. There are a significant amount of tests for MathematicalProgram, written in C++.  See [here](https://github.com/RobotLocomotion/drake/blob/master/solvers/test/mathematical_program_test.cc) but also other tests in that folder.  Occasionally you will still find a C++ feature that does not yet have pydrake bindings -- for those familiar with pybind, the bindings for MathematicalProgram are generated [here](https://github.com/RobotLocomotion/drake/blob/master/bindings/pydrake/solvers/mathematicalprogram_py.cc), and are demonstrated in numerous tests [here](https://github.com/RobotLocomotion/drake/tree/master/bindings/pydrake/solvers/test). Obviously though the features  demonstrated below all have pydrake bindings

### Okay, but how do I actually do an optimization problem?  

How do we translate something written on the board as a linear program, and write it down in code?

Here is a very simple example of a linear optimization problem:

\begin{align*}
        \min_{x} \ & x \\
        \text{s.t.} \ \ & x \ge 1 \\
\end{align*}

And the corresponding Mathematical Program code is below.

In [2]:
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
import numpy as np
import math

In [3]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddLinearCost(x[0]*1.0)
mp.AddLinearConstraint(x[0] >= 1)
result = Solve(mp)
print(result.is_success())
print(result.GetSolution(x))

True
[ 1.]


Note that written down slightly incorrectly, you will not get the answer you were looking for.  What is wrong about the two examples below?

In [4]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddLinearCost(x[0]*1.0)
result = Solve(mp)
print(result.is_success())
print(result.get_solution_result())

False
SolutionResult.kUnbounded


Note that you can get some information about the solution by printing out the result of `Solve(mp)`, which returns a [MathematicalProgramResult](https://drake.mit.edu/doxygen_cxx/classdrake_1_1solvers_1_1_mathematical_program_result.html).

In [5]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddLinearCost(x[0]*1.0)
mp.AddLinearConstraint(x[0] <= 1)
result = Solve(mp)
print(result.is_success())
print(result.get_solution_result())

False
SolutionResult.kUnbounded


Here's a slightly more complicated example.

In [6]:
mp = MathematicalProgram()
alpha = mp.NewContinuousVariables(1, "alpha")
mp.AddLinearCost(alpha[0]*1.0)
for xi in np.arange(-5*np.pi, 5*np.pi+np.pi/8, np.pi/8):
    mp.AddLinearConstraint(alpha[0] - math.cos(xi)**2 + math.sin(xi) >= 0)
    
result = Solve(mp)
print(result.GetSolution(alpha))

[ 1.23623682]


Note the MathematicalProgram is formulated in terms of "costs", and will minimize the objective function's costs when calling `Solve(mp)`.  How can we maximize functions? Just add a negative sign:

In [7]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddLinearCost(-x[0]*1.0)
mp.AddLinearConstraint(x[0] <= 4)
result = Solve(mp)
print(result.GetSolution(x))

[ 4.]


If, instead, we prefer quadratic cost functions, then this yields a quadratic program.

In [8]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddQuadraticCost((x[0]-3)**2)
result = Solve(mp)
print(result.GetSolution(x))

[ 3.]


Note that as above, problems with quadratic costs can be well formulated even without any constraints, which is not true of problems with only linear constraints.  And note that, because we didn't have any constraints, drake used a special purpose "solver", which in this case was just the closed-form solution.

Note that although there is no `QuadraticConstraint` in MathematicalProgram, it can generally handle a quadratic constraint, and many other different types of constraints, through `AddConstraint`, where inside the argument to the function is a symbolic formula of type `==`, `>=`, or `<=`.  This opens up MathematicalProgram to solve general nonlinear optimization problems.

In [9]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(2, "x")
mp.AddConstraint((x**2).sum() <= 1.0)
mp.AddLinearCost(x.sum())
result = Solve(mp)
print(result.GetSolution(x))

[-0.70710678 -0.70710678]


Alternatively you can even use many numpy operations, including `dot`:

In [10]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(2, "x")
mp.AddConstraint(x.dot(x) <= 1.)
mp.AddLinearCost(x.sum())
result = Solve(mp)
print(result.GetSolution(x))

[-0.70710678 -0.70710678]


Note that you can print out useful prints at many steps of interacting with Mathematical Program, for example:

In [11]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(2, "x")
print(type(x))
print(x)

<class 'numpy.ndarray'>
[Variable('x(0)', Continuous) Variable('x(1)', Continuous)]


In [12]:
y = x**2
print(y)

[<Expression "pow(x(0), 2)"> <Expression "pow(x(1), 2)">]
